## FineTuning para entrenamiento de clasificador de sentimientos.

### Objetivos:

* Entrenamiento de un modelo SBert que permita clasificar 6 tipo de sentimientos dentro de los que se incluyen {'sadness', 'joy', 'love', 'anger', 'fear', 'surprise'}.
* Evaluar con métricas clásicas el desempeño de la sintonía fina propuesta.

In [ ]:
### Carga del dataset
from datasets import load_dataset
import torch
dataset = load_dataset("dair-ai/emotion") 

### Visualización del formato del dataset.

In [ ]:
### Visualización de los datos :
print(dataset['train'][0])

{'text': 'i didnt feel humiliated', 'label': 0}


In [3]:
labels = dataset["train"].features["label"]
print(labels)

ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)


## Implementación de entrenamiento del modelo:

In [ ]:
import torch
import datasets
from transformers import DistilBertForSequenceClassification
from pandas import read_csv
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer


# Cargar el tokenizador de DistilBERT
model_name = "distilbert-base-uncased"

tokenizer = DistilBertTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Aplicar tokenización
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Quitar la columna "text", ya que no es necesaria después de la tokenización
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# Renombrar "label" a "labels" 
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Convertir a formato PyTorch
tokenized_datasets.set_format("torch")

### Carga del modelo:
model = None
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=6)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
        "precision": precision_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted")
    }

In [ ]:
import os
# Crear las carpetas si no existen
os.makedirs("./results", exist_ok=True)
os.makedirs("./logs", exist_ok=True)

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="C:/Users/dbasc/results", 
    logging_dir="C:/Users/dbasc/logs",   
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

c:\Users\dbasc\anaconda3\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\dbasc\AppData\Local\Temp\ipykernel_12904\501452110.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.098000,0.246916,0.925500,0.926701,0.930566,0.925500
2,0.058100,0.181795,0.935500,0.935249,0.935482,0.935500
3,0.078600,0.189957,0.942000,0.941907,0.942004,0.942000


TrainOutput(global_step=3000, training_loss=0.10886889019360145, metrics={'train_runtime': 6200.8372, 'train_samples_per_second': 7.741, 'train_steps_per_second': 0.484, 'total_flos': 1589722177536000.0, 'train_loss': 0.10886889019360145, 'epoch': 3.0})

In [17]:
model.save_pretrained("C:/Users/dbasc/emotion_model")
tokenizer.save_pretrained("C:/Users/dbasc/emotion_model")

('C:/Users/dbasc/emotion_model\\tokenizer_config.json',
 'C:/Users/dbasc/emotion_model\\special_tokens_map.json',
 'C:/Users/dbasc/emotion_model\\vocab.txt',
 'C:/Users/dbasc/emotion_model\\added_tokens.json')

## Testeo del modelo

In [58]:
eval_dataset = read_csv('test.csv')
eval_dataset

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [35]:
df = eval_dataset.copy()

In [45]:
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=128)

df['tokens'] = df['text'].apply(lambda x: tokenize_function(x))

df['input_ids'] = df['tokens'].apply(lambda x: x['input_ids'])
df['attention_mask'] = df['tokens'].apply(lambda x: x['attention_mask'])
df.drop(columns=['tokens'], inplace=True)


In [ ]:
evaluacion = df[['input_ids', 'attention_mask', 'label']]
eval_dataset = Dataset.from_pandas(evaluacion)
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


### Predicción del modelo según pesos entrenados

In [66]:
test_dataloader = DataLoader(eval_dataset, batch_size=8, collate_fn=data_collator)
model.eval()
predictions = []
real_labels = []

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    # Acá si el formato que dejaste los batches es distinto, adecuar funcion
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)
    # NO EDITAR    
    predictions.extend(predicted_labels.cpu().numpy())
    real_labels.extend(batch['labels'].cpu().numpy())


### Evaluación de Metricas del modelo

In [68]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

accuracy = accuracy_score(real_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(real_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

media = (accuracy + precision + recall + f1) / 4

print(f"Media: {media}")

Accuracy: 0.9285
Precision: 0.9288925099889768
Recall: 0.9285
F1 Score: 0.9282897881404893
Media: 0.9285455745323665
